In [ ]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score, recall_score,classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
# Para leer directamtente los datos
tf_idf = pd.read_csv("../data/sentiment_analysis/tf_idf/labelled_tf_idf.csv")
tf_idf.drop('Unnamed: 0',inplace=True,axis = 1)
tf_idf.head(2)

In [ ]:
# Create sets
X = tf_idf.drop(columns=["conversation_id","primary_label"], axis = 1)
y = tf_idf["primary_label"]

In [ ]:
from sklearn import svm
from sklearn.svm import SVC

svm_results = pd.DataFrame()
parameters_column = []
score_column = []
accuracy_column = []
recall_column = []
precision_column = []

for i in range(40):
    # Divide data in training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)
    #print(X_train.shape,X_test.shape)
    #print(y_train.shape, y_test.shape)

    # Validate parameters
    model = SVC()
    parameters = {
        'kernel':('linear', 'rbf'), 
        'C': np.logspace(-2, 3, 50),
        'gamma': np.logspace(-2, 3, 50)
        }
    clf = GridSearchCV(model,parameters,cv = 10, n_jobs = -1)
    clf.fit(X_train_std, y_train)

    best_parameters = clf.best_params_ 
    best_score = clf.best_score_ 
    best_estimator = clf.best_estimator_

    parameters_column.append(best_parameters)
    score_column.append(best_score)
    print(parameters)

    # Create model with best parameters
    best_estimator.fit(X_train_std,y_train)


    # Prediction
    y_pred = best_estimator.predict(X_test_std)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred,average='micro')
    recall = recall_score(y_test, y_pred,average='micro')
    print(accuracy,precision,recall)

    accuracy_column.append(accuracy)
    precision_column.append(precision)
    recall_column.append(recall)


svm_results["best_score"] = score_column
svm_results["best_parameters"] = parameters_column
svm_results["test_accuracy"] = accuracy_column
svm_results["test_micro_precision"] = precision_column
svm_results["test_micro_recall"] = recall_column
